<a href="https://colab.research.google.com/github/AlexeyK12/Data_scientist-Sberuniversity/blob/main/Uplift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uplift-моделирование

На этом занятии мы научимся строить uplift-модели при помощи библиотеки **scikit-uplift**.

Для этого мы:
- Подгрузим данные для моделирования
- Построим модель со сменой задачи
- Построим модель T-learn
- Построим модель S-learn
- Оценим качество наших моделей

**Uplift-моделирование** — это задача машинного обучения, в которой мы пытаемся найти объекты, которые при наличии на них влияния совершат целевое действие, а без отсутствия влияния целевое действие не совершат.

***Пример:*** мы продаем мобильные телефоны и выбираем, кому показать рекламу. Всем показать рекламу мы не можем, так как наш бюджет ограничен. Мы хотим найти таких клиентов, которые:
- Купят телефон, если увидят рекламу
- Не купят телефон, если рекламу не увидят

Для uplift-моделирования нам нужны 3 составляющие:
 - **Воздействие** (treatment, взаимодействие). Будем обозначать его через $t$. Это бинарная переменная, принимающая значение 0, если воздействие не совершено, и 1, если воздействие совершено
 - **Целевое действие** (target, таргет). Его будем обозначать через $y$. $y$ принимает значение 0, если клиент не совершил целевое действие, и 1, если совершил
 - **Признаки** о наших объектах (иногда мы их будем называть клиентами). Обозначим их через $x$

В нашем случае воздействие — это показ рекламы, а целевое действие — это покупка телефона, признаки — обычные признаки для клиентов: пол, возраст, история покупок.

Однако существует проблема: мы не можем одновременно показать и не показать рекламу телефона, поэтому мы должны работать не напрямую с целевыми действиями, а с их математическими ожиданиями:

$$ uplift = E(y | X, t=1) -  E(y | X, t=0) $$

Приступим к моделированию. Установим библиотеку **scikit-uplift**. Эта библиотека имеет интерфейс, похожий на интерфейс sklearn, но предназначена для построения uplift-моделей. Она содержит блок *datasets* с датасетами, блок *models* с моделями, а также блоки *metrics* и *viz* для оценки качества моделей.  

In [ ]:
! pip install scikit-uplift -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 774 kB/s 


Помимо библиотеки scikit-uplift, мы будем использовать стандартные библиотеки для анализа данных: pandas и sklearn.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklift.datasets import fetch_x5, fetch_lenta, clear_data_dir, fetch_megafon, fetch_hillstrom
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, TwoModels, ClassTransformation

from sklift.viz import plot_uplift_curve

# Подгрузка данных

Для работы мы будем использовать **Kevin Hillstrom Dataset**. В датасете содержатся данные о покупках в зависимости от того, получали ли клиенты email-рассылку или нет. Подгрузим данные при помощи функции fetch_hillstrom.

In [ ]:
data = fetch_hillstrom()

Hillstrom dataset:   0%|          | 0.00/443k [00:00<?, ?iB/s]

Посмотрим на наш датасет более детально.

In [ ]:
print(data['DESCR'])

Kevin Hillstrom Dataset: MineThatData

Data description
################

This is a copy of `MineThatData E-Mail Analytics And Data Mining Challenge dataset <https://blog.minethatdata.com/2008/03/minethatdata-e-mail-analytics-and-data.html>`_.

This dataset contains 64,000 customers who last purchased within twelve months.
The customers were involved in an e-mail test.

* 1/3 were randomly chosen to receive an e-mail campaign featuring Mens merchandise.
* 1/3 were randomly chosen to receive an e-mail campaign featuring Womens merchandise.
* 1/3 were randomly chosen to not receive an e-mail campaign.

During a period of two weeks following the e-mail campaign, results were tracked.
Your job is to tell the world if the Mens or Womens e-mail campaign was successful.

Fields
################

Historical customer attributes at your disposal include:

* Recency: Months since last purchase.
* History_Segment: Categorization of dollars spent in the past year.
* History: Actual dollar value spe

Для работы с uplift-моделями нам нужны данные о воздействиях на клиентов (treatment, t), данные о целевых действиях клиентов (target, y) и данные о характеристиках наших клиентов X. Проверим, что у нас есть все подходящие данные.

In [ ]:
X, y, t = data['data'],  data['target'], data['treatment']

In [ ]:
X.shape, y.shape, t.shape

((64000, 8), (64000,), (64000,))

Посмотрим на данные в виде таблицы, в которой указаны первые 5 строк с использованием функции head().

In [ ]:
X.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web


In [ ]:
t.sample(5)

3947       Mens E-Mail
48105      Mens E-Mail
15614      Mens E-Mail
58595        No E-Mail
21571    Womens E-Mail
Name: segment, dtype: object

In [ ]:
y.sample(5)

24511    0
27442    0
37124    1
3569     0
36720    0
Name: visit, dtype: int64

В данных содержатся бинарные переменные, которые необходимо трансформировать.

# preprocessing

Посмотрим, сколько уникальных значений встречается для каждой переменной в нашем датасете.

In [ ]:
X.nunique()

recency               12
history_segment        7
history            34833
mens                   2
womens                 2
zip_code               3
newbie                 2
channel                3
dtype: int64

In [ ]:
y.value_counts()

0    54606
1     9394
Name: visit, dtype: int64

In [ ]:
t.value_counts()

Womens E-Mail    21387
Mens E-Mail      21307
No E-Mail        21306
Name: segment, dtype: int64

Вектор со взаимодействиями представлен тремя видами: письмо для мужчин, письмо для женщин и отсутствие письма. В нашем упражнении мы не будем рассматривать отдельно мужчин и женщин, поэтому давайте преобразуем вектор t в бинарный, где 1 — наличие взаимодействия, а 0 — отсутствие взаимодействия.

In [ ]:
t = t.map({'Womens E-Mail':1, 'Mens E-Mail':1, 'No E-Mail':0})
t.head()

0    1
1    0
2    1
3    1
4    1
Name: segment, dtype: int64

Видов категориальных переменных в признаках не очень много. Для трансформации наших переменных используем метод **One Hot Encodding**. Но, чтобы не допустить утечки данных, сначала разобьем все данные на train и test.

In [ ]:
X_train, X_test, y_train, y_test, t_train, t_test = train_test_split(X, y, t, test_size=0.3, random_state=42)

В признаках X всего 3 категориальные переменные: history_segment, zip_code, channel. Трансформируем их и добавим к числовым переменным для X_train и X_test.

In [ ]:
cat_columns = ['history_segment', 'zip_code', 'channel']
enc = OneHotEncoder(sparse=False)

X_train_cat = enc.fit_transform(X_train[cat_columns])
X_train_cat = pd.DataFrame(X_train_cat,
                           index=X_train.index,
                           columns=enc.get_feature_names_out(cat_columns))

X_test_cat = enc.transform(X_test[cat_columns])
X_test_cat = pd.DataFrame(X_test_cat,
                          index=X_test.index,
                          columns=enc.get_feature_names_out(cat_columns))

X_train = pd.concat([X_train_cat, X_train.drop(cat_columns, axis=1)], axis=1)
X_test = pd.concat([X_test_cat, X_test.drop(cat_columns, axis=1)], axis=1)

In [ ]:
X_train.head()

,history_segment_1) $0 - $100,history_segment_2) $100 - $200,history_segment_3) $200 - $350,history_segment_4) $350 - $500,history_segment_5) $500 - $750,"history_segment_6) $750 - $1,000","history_segment_7) $1,000 +",zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,recency,history,mens,womens,newbie
9656,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7,434.35,1,0,1
63037,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,376.59,1,0,0
31405,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3,140.34,0,1,1
58088,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3,150.76,0,1,0
44344,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2,67.97,1,0,1


# Обучение моделей

Создадим переменную для записи результатов для различных моделей.

In [ ]:
model_predictions = dict()

## S-learner

Данную модель мы уже рассматривали в лонгриде. Ее основная идея в том, что мы обучаем базовую модель. Затем применяем  базовую модель сначала для данных, где указываем, что со всеми клиентами взаимодействовали (t = 1 для всех объектов), и просим вернуть вероятность y = 1 с помощью метода predict proba. На схеме эта часть обозначена через «Применение 1». Потом повторяем процедуру на этих же данных, но указываем, что взаимодействия не было (t = 0). На схеме ниже эта часть подписана «Применение 2». Разница между двумя векторами предсказаний и будет uplift.


![image](https://drive.google.com/uc?export=view&id=1x0hYltrTziBxLo1BJLYRM8kE_l2a6k_x)

Применим uplift-модель solo-model к нашим данным. В качестве базовой модели возьмем Random Forest. Запишем результаты в словарь model_predictions.

In [ ]:
model_name = 's_learner'

basic_model = RandomForestClassifier(random_state=42)
uplift_model = SoloModel(basic_model)
uplift_model = uplift_model.fit(X_train, y_train, t_train)

model_predictions[model_name] = uplift_model.predict(X_test)

## Изменение задачи

Суть данного подхода заключается к сведению задачи uplift к задаче бинарной классификации либо регрессии по признакам X. Рассмотрим, как можно заменить данный подход на задачу классификации:

$$ z =
\begin{equation*}
 \begin{cases}
   1 &\text{если y = 1 и t = 1 (подверглись воздействию и совершили целевое действие)}
   \\
   1 &\text{если y = 0 и t = 0 (не подвергались воздействию и не совершали целевое действие)}
   \\
   0 &\text{во всех остальных случаях}
 \end{cases}
\end{equation*}
$$

Далее для решения задачи мы можем применять модель бинарной классификации. В этом случае uplift — это уверенность нашей модели, что объект принадлежит классу 1, т. е. результат применения predict proba для класса 1.

Применим этот подход на наших данных. В качестве подобной базовой модели мы будем использовать модель RandomForestClassifier из библиотеки sklearn. В sсikit-uplift уже реализован функционал трансформации задачи, поэтому мы можем самостоятельно не изменять переменную y, а вызвать модель ClassTransformation и передать в нее модель базовую модель.

In [ ]:
model_name = 'task_transform'

basic_model = RandomForestClassifier(random_state=42)
uplift_model = ClassTransformation(basic_model)
uplift_model = uplift_model.fit(X_train, y_train, t_train)

model_predictions[model_name] = uplift_model.predict(X_test)

## T-learner

Подход с 2 моделями напоминает подход с одной моделью, однако вместо того, чтобы обучать одну базовую модель на всех данных, мы будем обучать 2 модели:
- $model_c$ для **контрольной** группы (тех, с кем yt взаимоделйствовали: t = 0)
- $model_c$ для **тестовой** группы (тех, с кем yt взаимоделйствовали: t = 1)


В нашем случае $n = n_c + n_t$,
где $n$ — размер выборки train, $n_c$ — размер контрольной (с кем не было взаимодействия) выборки для обучения, $n_t$ — размер тестовой (с кем было взаимодействие) выборки для обучения.
![image](https://drive.google.com/uc?export=view&id=1RG-1Djmoa_k194aHPITsz4wlNMgAYA-E)

Для получения вектора uplift применим наши модели на тестовых данных, как и в случае с Solo Model. Только в отличии от solo model мы не будем добавлять отдельный вектор взаимодействия к признакам, а будем использовать 2 независимые модели. Разница между двумя векторами predict proba и будет uplift для этой модели.

![image](https://drive.google.com/uc?export=view&id=1_4huBAFr7Yq4sK49Wuwx0RAe7k3K9NhN)

**ВНИМАНИЕ!**

$n_t$ — это размер тестовой подвыборки обучающей выборки. На ней мы обучаем модель $model_t$. В нашем коде нет отдельной переменной для хранения тестовой подвыборки обучающей выборки, так как она создается сама внутри модели TwoModels библиотеки scikit-uplift.

$n_{val}$ — размер валидационной выборки для проверки нашей модели. В коде для более привычного вида обозначено через X_test, y_test, t_test, как и в классическом машинном обучении.

Применим подход TwoModels для наших данных.

In [ ]:
model_name = 't_learner'

basic_model_control = RandomForestClassifier(random_state=42)
basic_model_test = RandomForestClassifier(random_state=42)

uplift_model = TwoModels(basic_model_test, basic_model_control, method='vanilla')
uplift_model = uplift_model.fit(X_train, y_train, t_train)

model_predictions[model_name] = uplift_model.predict(X_test)

# Оценка качества модели

Для оценки качества наших моделей мы будем применять метрику **uplift@k**. Данная метрика измеряется в диапазоне от –1 до 1, где 1 — самый лучший вариант, а –1 — абсолютно неверная работающая модель. Если значение метрики 0, то наша модель работает эквивалентно случайной модели. Данная модель крайне редко получает высокие значения, близкие к 1, поэтому результат больше 0.05 уже считается неслучайным.  

Суть данного подхода в следующем:
1. Берем k объектов с самым высоким uplift из нашей отложенной выборки для проверки. k — это или количество объектов, или доля объектов от общей выборки
2. Делим получившуюся подвыборку на контроль (t = 0) и тест (t = 1)
3. Рассчитываем средний таргет $y$ для каждой группы отдельно
4. Находим разницу

$$ uplift@k = \bar{y}_{k \space test} - \bar{y}_{k \space control} $$

$$ \bar{y}_k = \frac{1}{n} \sum_{i=1}^{k}{uplift_i,}  $$

где $uplift_i$ — $i$-й объект uplift-вектора, отсортированного в порядке убывания (лучшие объекты идут вначале). Данный подход называется **uplift@k** общий, или overall.

Существует также второй способ рассчитать **uplift@k**. Он называется подход по группам. Разница с overall-подходом в том, что мы в подходе по группам мы сначала разбиваем данные не тест и контроль, а уже потом отбираем лучшие k объектов.


Более подробно про эту и другие метрики вы прочитать в туториал на Habr: https://habr.com/ru/company/ru_mts/blog/538934.

Рассчитаем оба варианта метрики uplift@k для наших данных. В качестве k будем брать 20 % наших лучших предсказаний.

In [ ]:
results = dict()

for model_name, preds in model_predictions.items():
    up_k_best_gr = uplift_at_k(y_true=y_test, uplift=preds, treatment=t_test,
                strategy='by_group', k=0.2)

    up_k_best_over = uplift_at_k(y_true=y_test, uplift=preds, treatment=t_test,
                strategy='overall', k=0.2)

    results[model_name] = [up_k_best_gr, up_k_best_over]

print("uplift@k")
pd.DataFrame(results.values(), index=results.keys(), columns=['up@k_gr', 'up@k_over'])

uplift@k


,up@k_gr,up@k_over
s_learner,0.052072,0.052598
task_transform,0.071552,0.071409
t_learner,0.060676,0.058424


Как видно из таблицы с результатами, на нашем датасете лучше всего себя проявил подход с подменой задачи. Однако так бывает не всегда, и обычно лучшие результаты показывает модель T-learner.

# Выводы

Мы изучили модели для uplift-моделирования. Основная идея в uplift-моделировании заключается в использовании одной или двух моделей для независимых выборок. Но иногда можно свести задачу к задаче классификации в терминах uplift, что не всегда дает хороший результат.

В случаях, когда у нас есть данные о воздействиях на клиентов и совершении ими целевых действий, лучше решать задачу определения, на кого оказывать воздействие в рамках задачи uplift-моделирования. Однако когда подобных данных нет, то можно свести задачу к задаче бинарной классификации, но результат моделирования будет менее точным по сравнению с uplift-подходами.